In [8]:
import oda_api.token 
import logging
import numpy as np
from oda_api.api import DispatcherAPI
from oda_api.plot_tools import OdaImage
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from matplotlib.patches import Circle

In [6]:
# token = ''
# token = getpass.getpass('Insert the token')
token = oda_api.token.discover_token() # get token stored locally

In [7]:
logging.getLogger().setLevel(logging.INFO) # WARNING, INFO or DEBUG
logging.getLogger('oda_api').addHandler(logging.StreamHandler())

Import the ScWs from the saved file.

In [56]:
scw_ids = []
scw_versions = []
scw_start_times = []
scw_end_times = []

with open("2003-01-01_2004-01-01.txt", "r") as f:
    next(f)
    for line in f:
        parts = line.strip().split(", ")
        scw_ids.append(parts[0])
        scw_versions.append(parts[1])
        scw_start_times.append(float(parts[2]))  
        scw_end_times.append(float(parts[3]))  


In [ ]:
disp = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", instrument="mock")

par_dict = {
# "E1_keV": "15",
# "E2_keV": "30",
# "T_format": "isot",
"detection_threshold": "5",
"instrument": "isgri",
"osa_version": "OSA10.2",
"product": "isgri_image",
"product_type": "Real",
"scw_list": ["027300510010.001"],
'token': token
}
# "integral_data_rights": "all-private" had to be removed as didn't have the rights

data_collection = disp.get_product(**par_dict)
data_collection.show()
data_collection.mosaic_image_0_mosaic.show()

print(data_collection.mosaic_image_0_mosaic.show_meta())

In [ ]:
im=OdaImage(data_collection)
_ = im.show()
im.write_fits('')

In [ ]:
from astropy.wcs import WCS
from matplotlib.patches import Circle

fits_file = "027300510010.001mosaic.fits"

# Jupiter
ra_deg = ra_values[0]
dec_deg = dec_values[0]

with fits.open(fits_file) as hdul:
    image_data = hdul[2].data 
    wcs = WCS(hdul[2].header)  

x, y = wcs.all_world2pix(ra_deg, dec_deg, 1)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection=wcs)
im = ax.imshow(np.log10(image_data), cmap='plasma', origin='lower')

ax.set_xlabel('RA')
ax.set_ylabel('DEC')

pixel_scale = np.abs(wcs.wcs.cdelt[0]) * 3600  # pixel size in arcseconds
circle_radius_pixels = 30 * 50 / pixel_scale / 2  # radius in pixels

circle = Circle((x, y), circle_radius_pixels, edgecolor='black', facecolor='none', lw=2)
ax.add_patch(circle)

plt.colorbar(im, label="Flux")
plt.show()